In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob
import pickle
import xgboost as xgb
import numpy as np
import pandas as pd

In [12]:
df_ethnicity_test = pd.read_csv("etnicity_labels_test.csv")
display(df_ethnicity_test)
df_ethnicity_train = pd.read_csv("etnicity_labels_train.csv")
display(df_ethnicity_train)

,Ethnicity
0,2
1,2
2,3
3,2
4,3
...,...
1995,2
1996,2
1997,1
1998,1


,Ethnicity
0,2
1,2
2,2
3,2
4,2
...,...
5995,2
5996,3
5997,2
5998,2


In [2]:
def createcsv(dataframe, filename): #make sure to include .csv at the end of the filename
    dataframe.to_csv(filename,index=False)

In [3]:
csv_file = "complete_CSV_Train.csv"
complete_csv_train = pd.read_csv("complete_CSV_Train.csv")
csv_file = "complete_CSV_validation.csv"
complete_csv_test = pd.read_csv("complete_CSV_validation.csv")

,File_name,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,...,right_eye_right_min,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Interview
0,--Ymqszjv54.004.mp4,2.00,3.32,1.40,2.89,1.16,2.45,1.10,2.02,2.04,...,0.000000,42.612674,33.860635,0.000000,0.544444,0.368932,0.317757,0.549451,0.322917,0.401869
1,-55DRRMTppE.005.mp4,2.81,1.76,1.36,1.51,0.95,1.69,0.59,0.24,1.96,...,8.402381,43.971127,41.850294,38.129516,0.733333,0.737864,0.738318,0.769231,0.687500,0.757009
2,-AmMDnVl4s8.003.mp4,1.16,1.19,1.40,1.20,1.84,2.09,0.84,2.18,3.12,...,7.144928,45.553156,42.081365,37.701194,0.788889,0.757282,0.766355,0.824176,0.843750,0.738318
3,-DOqN0d8KHw.003.mp4,1.51,1.22,0.43,2.27,1.23,1.27,1.43,1.84,2.78,...,7.100000,64.829777,55.926806,45.819319,0.533333,0.495146,0.551402,0.648352,0.447917,0.383178
4,-fqiCqZtgYs.002.mp4,3.42,2.76,3.08,1.83,0.66,2.67,1.15,1.44,1.29,...,4.767599,69.892775,51.245649,28.549081,0.577778,0.320388,0.271028,0.263736,0.333333,0.242991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,_LWrd9t4oZw.003.mp4,2.20,1.38,0.00,1.47,1.65,2.53,1.13,1.44,3.14,...,5.408327,70.058904,62.175441,52.909451,0.733333,0.611650,0.682243,0.582418,0.645833,0.598131
5996,_S_bWJwqsmM.004.mp4,5.00,5.00,1.06,4.91,1.76,5.00,1.01,3.62,1.66,...,3.900000,42.507058,38.928617,7.324616,0.500000,0.300971,0.429907,0.450549,0.364583,0.299065
5997,_X6_C_Oy9VA.005.mp4,1.99,1.36,0.09,0.79,1.00,1.15,1.14,2.30,1.30,...,2.973214,69.468338,61.760180,47.897912,0.544444,0.504854,0.579439,0.736264,0.489583,0.551402
5998,_ZjEr8ybbFM.000.mp4,3.63,1.93,0.98,2.16,1.48,3.82,3.71,2.80,1.84,...,4.119466,44.171484,33.500963,3.701351,0.588889,0.407767,0.457944,0.450549,0.468750,0.392523


,File_name,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,...,right_eye_right_min,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Interview
0,-6otZ7M-Mro.003.mp4,1.52,0.85,2.34,0.75,1.62,1.54,0.65,2.61,2.07,...,1.649242,67.846076,60.047009,53.022637,0.666667,0.728155,0.710280,0.681319,0.552083,0.654206
1,-6otZ7M-Mro.005.mp4,3.95,1.11,2.25,2.25,0.95,1.91,0.85,1.90,0.75,...,1.552417,62.368261,58.543807,42.257307,0.600000,0.728155,0.523364,0.626374,0.635417,0.644860
2,-8asrRvfJWA.001.mp4,3.08,1.72,0.00,1.40,0.52,1.16,0.75,0.87,2.21,...,5.600893,43.683521,39.915656,30.364618,0.611111,0.582524,0.485981,0.483516,0.500000,0.457944
3,-9BZ8A9U7TE.002.mp4,0.93,1.18,0.00,1.83,0.00,0.30,1.23,0.00,0.21,...,5.295281,58.649126,53.179319,46.608690,0.544444,0.417476,0.579439,0.494505,0.468750,0.364486
4,-agCXYgb7pI.000.mp4,3.77,3.27,1.81,1.04,0.28,1.70,0.60,1.75,0.85,...,3.805260,34.973847,33.081941,30.302640,0.700000,0.728155,0.457944,0.615385,0.552083,0.607477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,_uNup91ZYw0.000.mp4,2.68,1.39,1.07,1.03,2.16,2.42,0.52,3.15,2.66,...,4.501111,30.850770,27.635797,21.800229,0.744444,0.631068,0.700935,0.769231,0.760417,0.682243
1996,_VsE38xSmdM.003.mp4,2.58,1.30,1.26,1.24,1.73,2.35,0.82,2.29,2.37,...,6.806614,43.960209,40.236056,36.577589,0.866667,0.718447,0.831776,0.714286,0.843750,0.794393
1997,_ZAqTHlu2ts.000.mp4,1.32,0.83,2.19,0.83,0.94,0.53,0.91,1.40,1.86,...,8.668333,43.532976,40.535075,37.558754,0.211111,0.213592,0.158879,0.252747,0.125000,0.205607
1998,_ZAqTHlu2ts.002.mp4,1.57,1.11,1.92,2.10,1.31,1.06,1.24,1.48,2.30,...,6.958448,44.261044,40.494462,37.348360,0.366667,0.271845,0.233645,0.362637,0.218750,0.299065


In [4]:
df_etn = pd.read_excel("eth_gender_anno_all.xlsx")

In [5]:
display(df_etn)

,VideoName,YouTubeID,Ethnicity,Gender
0,_01AyUz9J9I.000.mp4,_01AyUz9J9I,2,1
1,_01AyUz9J9I.002.mp4,_01AyUz9J9I,2,1
2,_01AyUz9J9I.003.mp4,_01AyUz9J9I,2,1
3,_01AyUz9J9I.005.mp4,_01AyUz9J9I,2,1
4,_01AyUz9J9I.001.mp4,_01AyUz9J9I,2,1
...,...,...,...,...
9995,zyGz_H1UTnQ.000.mp4,zyGz_H1UTnQ,2,1
9996,zyGz_H1UTnQ.001.mp4,zyGz_H1UTnQ,2,1
9997,ZYmZ6fOnEVg.000.mp4,ZYmZ6fOnEVg,2,2
9998,ZYrFfvp7_Mo.000.mp4,ZYrFfvp7_Mo,3,1


In [6]:
test_df = complete_csv.head(10)

NameError: name 'complete_csv' is not defined

In [56]:
display(test_df)

,File_name,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,...,right_eye_right_min,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Interview
0,--Ymqszjv54.004.mp4,2.00,3.32,1.40,2.89,1.16,2.45,1.10,2.02,2.04,...,0.000000,42.612674,33.860635,0.000000,0.544444,0.368932,0.317757,0.549451,0.322917,0.401869
1,-55DRRMTppE.005.mp4,2.81,1.76,1.36,1.51,0.95,1.69,0.59,0.24,1.96,...,8.402381,43.971127,41.850294,38.129516,0.733333,0.737864,0.738318,0.769231,0.687500,0.757009
2,-AmMDnVl4s8.003.mp4,1.16,1.19,1.40,1.20,1.84,2.09,0.84,2.18,3.12,...,7.144928,45.553156,42.081365,37.701194,0.788889,0.757282,0.766355,0.824176,0.843750,0.738318
3,-DOqN0d8KHw.003.mp4,1.51,1.22,0.43,2.27,1.23,1.27,1.43,1.84,2.78,...,7.100000,64.829777,55.926806,45.819319,0.533333,0.495146,0.551402,0.648352,0.447917,0.383178
4,-fqiCqZtgYs.002.mp4,3.42,2.76,3.08,1.83,0.66,2.67,1.15,1.44,1.29,...,4.767599,69.892775,51.245649,28.549081,0.577778,0.320388,0.271028,0.263736,0.333333,0.242991
5,-Gl98Jn45Fs.003.mp4,1.03,1.33,1.96,0.96,1.45,0.15,0.78,2.13,1.89,...,5.603570,32.250271,30.009379,26.743410,0.655556,0.572816,0.532710,0.505495,0.677083,0.635514
6,-IK--4uz5ZY.004.mp4,2.45,1.41,0.27,3.03,0.89,2.24,0.65,0.46,1.60,...,6.873136,80.674407,71.242003,64.625073,0.811111,0.689320,0.747664,0.747253,0.677083,0.785047
7,-R2SZu3SYgM.000.mp4,2.51,1.26,0.95,1.72,0.78,2.28,1.43,1.01,1.37,...,6.303174,64.709350,54.368363,41.246212,0.577778,0.194175,0.224299,0.197802,0.166667,0.214953
8,-utrsarZeIY.001.mp4,2.31,2.09,2.86,0.98,1.71,0.49,0.89,2.41,2.22,...,4.904080,60.705272,50.748524,29.904013,0.622222,0.572816,0.504673,0.582418,0.458333,0.551402
9,-VTqcHNgH7M.003.mp4,1.33,1.01,0.00,0.40,0.00,0.00,0.91,0.22,0.06,...,5.103920,34.205263,31.613267,27.961402,0.622222,0.592233,0.448598,0.560440,0.677083,0.532710


In [7]:
def add_ethnicity(df1, df2): #df1 is complete df df2 is ethnicity df
    outputlist = []
    count = 1
    for i in range(0,len(df1)):
        print(count, "out of 6000")
        count += 1
        for j in range(0,len(df2)):
            if df1.iloc[i,0] == df2.iloc[j,0]:
                outputlist.append(df2.iloc[j,2])
    outputdf = pd.DataFrame(outputlist, columns=['Ethnicity'])
    return outputdf
            

In [8]:
df_ethnicity_train = add_ethnicity(complete_csv_train, df_etn)
display(df_ethnicity_train)

1 out of 6000
2 out of 6000
3 out of 6000
4 out of 6000
5 out of 6000
6 out of 6000
7 out of 6000
8 out of 6000
9 out of 6000
10 out of 6000
11 out of 6000
12 out of 6000
13 out of 6000
14 out of 6000
15 out of 6000
16 out of 6000
17 out of 6000
18 out of 6000
19 out of 6000
20 out of 6000
21 out of 6000
22 out of 6000
23 out of 6000
24 out of 6000
25 out of 6000
26 out of 6000
27 out of 6000
28 out of 6000
29 out of 6000
30 out of 6000
31 out of 6000
32 out of 6000
33 out of 6000
34 out of 6000
35 out of 6000
36 out of 6000
37 out of 6000
38 out of 6000
39 out of 6000
40 out of 6000
41 out of 6000
42 out of 6000
43 out of 6000
44 out of 6000
45 out of 6000
46 out of 6000
47 out of 6000
48 out of 6000
49 out of 6000
50 out of 6000
51 out of 6000
52 out of 6000
53 out of 6000
54 out of 6000
55 out of 6000
56 out of 6000
57 out of 6000
58 out of 6000
59 out of 6000
60 out of 6000
61 out of 6000
62 out of 6000
63 out of 6000
64 out of 6000
65 out of 6000
66 out of 6000
67 out of 6000
68 o

520 out of 6000
521 out of 6000
522 out of 6000
523 out of 6000
524 out of 6000
525 out of 6000
526 out of 6000
527 out of 6000
528 out of 6000
529 out of 6000
530 out of 6000
531 out of 6000
532 out of 6000
533 out of 6000
534 out of 6000
535 out of 6000
536 out of 6000
537 out of 6000
538 out of 6000
539 out of 6000
540 out of 6000
541 out of 6000
542 out of 6000
543 out of 6000
544 out of 6000
545 out of 6000
546 out of 6000
547 out of 6000
548 out of 6000
549 out of 6000
550 out of 6000
551 out of 6000
552 out of 6000
553 out of 6000
554 out of 6000
555 out of 6000
556 out of 6000
557 out of 6000
558 out of 6000
559 out of 6000
560 out of 6000
561 out of 6000
562 out of 6000
563 out of 6000
564 out of 6000
565 out of 6000
566 out of 6000
567 out of 6000
568 out of 6000
569 out of 6000
570 out of 6000
571 out of 6000
572 out of 6000
573 out of 6000
574 out of 6000
575 out of 6000
576 out of 6000
577 out of 6000
578 out of 6000
579 out of 6000
580 out of 6000
581 out of 6000
582 out 

KeyboardInterrupt: 

In [72]:
createcsv(df_ethnicity_train, "etnicity_labels_train.csv")

In [70]:
df_ethnicity_test = add_ethnicity(complete_csv_test, df_etn)
display(df_ethnicity_test)

1 out of 6000
2 out of 6000
3 out of 6000
4 out of 6000
5 out of 6000
6 out of 6000
7 out of 6000
8 out of 6000
9 out of 6000
10 out of 6000
11 out of 6000
12 out of 6000
13 out of 6000
14 out of 6000
15 out of 6000
16 out of 6000
17 out of 6000
18 out of 6000
19 out of 6000
20 out of 6000
21 out of 6000
22 out of 6000
23 out of 6000
24 out of 6000
25 out of 6000
26 out of 6000
27 out of 6000
28 out of 6000
29 out of 6000
30 out of 6000
31 out of 6000
32 out of 6000
33 out of 6000
34 out of 6000
35 out of 6000
36 out of 6000
37 out of 6000
38 out of 6000
39 out of 6000
40 out of 6000
41 out of 6000
42 out of 6000
43 out of 6000
44 out of 6000
45 out of 6000
46 out of 6000
47 out of 6000
48 out of 6000
49 out of 6000
50 out of 6000
51 out of 6000
52 out of 6000
53 out of 6000
54 out of 6000
55 out of 6000
56 out of 6000
57 out of 6000
58 out of 6000
59 out of 6000
60 out of 6000
61 out of 6000
62 out of 6000
63 out of 6000
64 out of 6000
65 out of 6000
66 out of 6000
67 out of 6000
68 o

520 out of 6000
521 out of 6000
522 out of 6000
523 out of 6000
524 out of 6000
525 out of 6000
526 out of 6000
527 out of 6000
528 out of 6000
529 out of 6000
530 out of 6000
531 out of 6000
532 out of 6000
533 out of 6000
534 out of 6000
535 out of 6000
536 out of 6000
537 out of 6000
538 out of 6000
539 out of 6000
540 out of 6000
541 out of 6000
542 out of 6000
543 out of 6000
544 out of 6000
545 out of 6000
546 out of 6000
547 out of 6000
548 out of 6000
549 out of 6000
550 out of 6000
551 out of 6000
552 out of 6000
553 out of 6000
554 out of 6000
555 out of 6000
556 out of 6000
557 out of 6000
558 out of 6000
559 out of 6000
560 out of 6000
561 out of 6000
562 out of 6000
563 out of 6000
564 out of 6000
565 out of 6000
566 out of 6000
567 out of 6000
568 out of 6000
569 out of 6000
570 out of 6000
571 out of 6000
572 out of 6000
573 out of 6000
574 out of 6000
575 out of 6000
576 out of 6000
577 out of 6000
578 out of 6000
579 out of 6000
580 out of 6000
581 out of 6000
582 out 

1031 out of 6000
1032 out of 6000
1033 out of 6000
1034 out of 6000
1035 out of 6000
1036 out of 6000
1037 out of 6000
1038 out of 6000
1039 out of 6000
1040 out of 6000
1041 out of 6000
1042 out of 6000
1043 out of 6000
1044 out of 6000
1045 out of 6000
1046 out of 6000
1047 out of 6000
1048 out of 6000
1049 out of 6000
1050 out of 6000
1051 out of 6000
1052 out of 6000
1053 out of 6000
1054 out of 6000
1055 out of 6000
1056 out of 6000
1057 out of 6000
1058 out of 6000
1059 out of 6000
1060 out of 6000
1061 out of 6000
1062 out of 6000
1063 out of 6000
1064 out of 6000
1065 out of 6000
1066 out of 6000
1067 out of 6000
1068 out of 6000
1069 out of 6000
1070 out of 6000
1071 out of 6000
1072 out of 6000
1073 out of 6000
1074 out of 6000
1075 out of 6000
1076 out of 6000
1077 out of 6000
1078 out of 6000
1079 out of 6000
1080 out of 6000
1081 out of 6000
1082 out of 6000
1083 out of 6000
1084 out of 6000
1085 out of 6000
1086 out of 6000
1087 out of 6000
1088 out of 6000
1089 out of 60

1513 out of 6000
1514 out of 6000
1515 out of 6000
1516 out of 6000
1517 out of 6000
1518 out of 6000
1519 out of 6000
1520 out of 6000
1521 out of 6000
1522 out of 6000
1523 out of 6000
1524 out of 6000
1525 out of 6000
1526 out of 6000
1527 out of 6000
1528 out of 6000
1529 out of 6000
1530 out of 6000
1531 out of 6000
1532 out of 6000
1533 out of 6000
1534 out of 6000
1535 out of 6000
1536 out of 6000
1537 out of 6000
1538 out of 6000
1539 out of 6000
1540 out of 6000
1541 out of 6000
1542 out of 6000
1543 out of 6000
1544 out of 6000
1545 out of 6000
1546 out of 6000
1547 out of 6000
1548 out of 6000
1549 out of 6000
1550 out of 6000
1551 out of 6000
1552 out of 6000
1553 out of 6000
1554 out of 6000
1555 out of 6000
1556 out of 6000
1557 out of 6000
1558 out of 6000
1559 out of 6000
1560 out of 6000
1561 out of 6000
1562 out of 6000
1563 out of 6000
1564 out of 6000
1565 out of 6000
1566 out of 6000
1567 out of 6000
1568 out of 6000
1569 out of 6000
1570 out of 6000
1571 out of 60

1995 out of 6000
1996 out of 6000
1997 out of 6000
1998 out of 6000
1999 out of 6000
2000 out of 6000


,Ethnicity
0,2
1,2
2,3
3,2
4,3
...,...
1995,2
1996,2
1997,1
1998,1


In [71]:
createcsv(df_ethnicity_test, "etnicity_labels_test.csv")

In [ ]:
df_ethnicity_train = add_ethnicity(complete_csv_train, df_etn)
display(df_ethnicity_train)

In [59]:
df_ethnicity = pd.DataFrame(list_ethnicity, columns=['Ethnicity'])

,Ethnicity
0,2
1,2
2,2
3,2
4,2
5,2
6,2
7,2
8,2
9,2


In [13]:
def add_etnicity (csv_file):
    df = pd.read_csv(csv_file)
    #display(df)
    x_array = df.iloc[:,1:-6]
    #print(df1)
    y_array = df.iloc[:,230:]
    return x_array, y_array

In [15]:
def structure_df(df1,df2): #df1 = main df including labels df 2 = ethnicity
    df_first = df1.iloc[:,1:-6]
    df_second = df2
    df_third = df1.iloc[:,-6:]
    outputdf1 = pd.concat([df_first, df_second], axis=1).reindex(df1.index)
    outputdf2 = pd.concat([outputdf1, df_third], axis=1).reindex(df1.index)
    return outputdf1, outputdf2

In [17]:
df_etn_no_labels_train, df_etn_labels_train = structure_df(complete_csv_train, df_ethnicity_train)

In [19]:
df_etn_no_labels_test, df_etn_labels_test = structure_df(complete_csv_test, df_ethnicity_test)

In [171]:
df_etn_no_labels_test[0:1]

,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,AU14_r_max,...,right_eye_left_max,right_eye_left_mean,right_eye_left_min,right_eye_right_max,right_eye_right_mean,right_eye_right_min,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Ethnicity
0,1.52,0.85,2.34,0.75,1.62,1.54,0.65,2.61,2.07,2.17,...,18.558556,12.945213,3.569314,17.396839,12.038912,1.649242,67.846076,60.047009,53.022637,2


In [21]:
def devide_etn (df): #with labels
    dfoutput1 = pd.DataFrame(data=[])
    dfoutput2 = pd.DataFrame(data=[])
    dfoutput3 = pd.DataFrame(data=[])
    for i in range(0, len(df)):

        if df.iloc[i][224] == 1:
            dfoutput1 = pd.concat([dfoutput1, df[i:i+1]])

        elif df.iloc[i][224] == 2:
            dfoutput2 = pd.concat([dfoutput2, df[i:i+1]])

        else:
            dfoutput3 = pd.concat([dfoutput3, df[i:i+1]])

    return dfoutput1, dfoutput2, dfoutput3

In [23]:
Asia_train, Caucasian_train, African_American_train = devide_etn(df_etn_labels_train)

In [207]:
Asia_train_and_test = pd.concat([Asia_train, Asia_test])
display(Asia_train_and_test)

,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,AU14_r_max,...,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Ethnicity,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Interview
65,2.21,1.80,1.14,1.39,2.01,2.20,0.69,2.62,2.54,2.70,...,71.759111,61.016768,39.214921,1,0.622222,0.407767,0.448598,0.516484,0.552083,0.448598
72,2.15,1.67,1.40,1.01,2.09,3.44,1.14,2.36,2.38,1.74,...,52.155249,48.701332,45.862512,1,0.633333,0.514563,0.616822,0.516484,0.437500,0.532710
79,2.58,1.50,1.94,2.72,2.21,3.46,0.82,1.53,1.91,1.63,...,59.605369,50.843154,41.793899,1,0.777778,0.553398,0.570093,0.626374,0.708333,0.570093
285,3.29,1.61,2.62,1.60,1.81,3.24,1.69,1.60,0.56,1.04,...,59.434838,55.029221,39.840683,1,0.500000,0.640777,0.485981,0.593407,0.458333,0.514019
327,2.15,1.15,0.39,0.80,1.69,0.86,1.03,0.38,2.58,1.46,...,60.104076,55.766753,50.683035,1,0.600000,0.679612,0.467290,0.604396,0.479167,0.560748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,2.26,1.89,0.00,1.02,1.99,3.75,0.30,1.24,1.78,2.91,...,93.754627,83.014919,78.606361,1,0.422222,0.495146,0.476636,0.494505,0.364583,0.457944
1939,3.46,2.36,2.29,1.22,1.37,2.83,0.73,1.66,1.13,2.52,...,63.965147,50.338239,33.630046,1,0.455556,0.281553,0.485981,0.395604,0.479167,0.401869
1997,1.32,0.83,2.19,0.83,0.94,0.53,0.91,1.40,1.86,1.73,...,43.532976,40.535075,37.558754,1,0.211111,0.213592,0.158879,0.252747,0.125000,0.205607
1998,1.57,1.11,1.92,2.10,1.31,1.06,1.24,1.48,2.30,1.58,...,44.261044,40.494462,37.348360,1,0.366667,0.271845,0.233645,0.362637,0.218750,0.299065


In [208]:
Caucasian_train_and_test = pd.concat([Caucasian_train, Caucasian_test])
display(Caucasian_train_and_test)

,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,AU14_r_max,...,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Ethnicity,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Interview
0,2.00,3.32,1.40,2.89,1.16,2.45,1.10,2.02,2.04,2.28,...,42.612674,33.860635,0.000000,2,0.544444,0.368932,0.317757,0.549451,0.322917,0.401869
1,2.81,1.76,1.36,1.51,0.95,1.69,0.59,0.24,1.96,1.13,...,43.971127,41.850294,38.129516,2,0.733333,0.737864,0.738318,0.769231,0.687500,0.757009
2,1.16,1.19,1.40,1.20,1.84,2.09,0.84,2.18,3.12,1.24,...,45.553156,42.081365,37.701194,2,0.788889,0.757282,0.766355,0.824176,0.843750,0.738318
3,1.51,1.22,0.43,2.27,1.23,1.27,1.43,1.84,2.78,2.07,...,64.829777,55.926806,45.819319,2,0.533333,0.495146,0.551402,0.648352,0.447917,0.383178
4,3.42,2.76,3.08,1.83,0.66,2.67,1.15,1.44,1.29,2.04,...,69.892775,51.245649,28.549081,2,0.577778,0.320388,0.271028,0.263736,0.333333,0.242991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,3.09,2.84,0.88,0.74,0.78,1.12,1.05,1.18,1.45,1.56,...,54.657570,51.014717,43.884963,2,0.466667,0.592233,0.392523,0.516484,0.343750,0.514019
1993,1.62,1.94,3.10,0.62,2.37,3.16,0.59,2.62,2.78,3.51,...,47.056137,42.135737,35.723942,2,0.566667,0.475728,0.523364,0.527473,0.510417,0.532710
1994,1.95,1.58,0.91,1.19,0.00,0.65,0.65,0.38,0.00,0.00,...,37.184136,33.119999,29.361539,2,0.622222,0.582524,0.523364,0.571429,0.697917,0.598131
1995,2.68,1.39,1.07,1.03,2.16,2.42,0.52,3.15,2.66,2.66,...,30.850770,27.635797,21.800229,2,0.744444,0.631068,0.700935,0.769231,0.760417,0.682243


In [214]:
African_American_train_and_test = pd.concat([African_American_train, African_American_test])
display(African_American_train_and_test)

,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,AU14_r_max,...,right_eye_corner_max,right_eye_corner_mean,right_eye_corner_min,Ethnicity,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Interview
14,1.34,1.91,0.96,1.45,1.20,0.79,0.93,0.81,1.60,2.48,...,54.350345,49.138598,41.580043,3,0.477778,0.368932,0.345794,0.384615,0.406250,0.429907
17,2.04,1.33,0.82,0.73,3.08,3.94,3.29,2.48,3.20,3.41,...,50.666162,38.108873,31.391878,3,0.355556,0.262136,0.317757,0.329670,0.437500,0.261682
18,2.90,1.48,0.34,1.16,1.25,1.30,1.62,1.20,1.18,3.78,...,40.848990,37.052066,32.339604,3,0.422222,0.300971,0.327103,0.417582,0.260417,0.252336
19,3.62,2.18,0.07,1.13,1.58,1.80,0.99,1.85,1.97,4.50,...,41.105961,36.897925,32.770261,3,0.377778,0.271845,0.299065,0.296703,0.260417,0.205607
46,2.68,1.34,0.57,1.81,2.89,3.08,0.47,2.14,3.39,3.74,...,69.346161,60.283978,48.208402,3,0.333333,0.300971,0.401869,0.362637,0.250000,0.261682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,2.71,2.09,1.10,2.95,2.29,3.28,0.87,2.95,3.09,3.27,...,61.968782,48.012874,29.162476,3,0.588889,0.339806,0.504673,0.450549,0.541667,0.532710
1916,2.80,3.19,0.08,2.18,1.01,1.99,0.59,1.03,2.35,2.36,...,72.379279,53.166608,40.133029,3,0.533333,0.543689,0.504673,0.560440,0.458333,0.551402
1917,1.37,1.31,0.50,0.94,1.17,2.47,0.91,1.03,1.93,2.37,...,56.602915,48.823628,37.657934,3,0.522222,0.388350,0.485981,0.626374,0.562500,0.551402
1952,2.48,1.82,2.03,1.76,1.32,1.01,0.82,3.38,1.87,3.60,...,37.647975,35.208819,31.101447,3,0.622222,0.281553,0.355140,0.571429,0.416667,0.457944


In [211]:
createcsv(Asia_train_and_test, "Asia_train_and_test.csv")

In [212]:
createcsv(Caucasian_train_and_test, "Caucasion_train_and_test.csv")

In [215]:
createcsv(African_American_train_and_test, "African_Amarican_train_and_test.csv")

In [26]:
createcsv(Asia_train, "Asia_train.csv")
createcsv(Caucasian_train, "Caucasion_train.csv")
createcsv(African_American_train, "African_American_train.csv")

createcsv(Asia_test, "Asia_test.csv")
createcsv(Caucasian_test, "Caucasian_test.csv")
createcsv(African_American_test, "African_American_test.csv")